# 01 IMPORTS

In [24]:
import pandas as pd
import glob
import numpy as np
import matplotlib.pyplot as plt
import time
from datetime import datetime

import warnings # necessary b/c pandas & statsmodels datetime issue
warnings.simplefilter(action="ignore")

# 02 DATA IMPORT

In [25]:
df = pd.read_csv('C:\\Users\\benro\\OneDrive\\Documents\\EODData\\StockProject_New\\Combined1MinData_042022_062022.csv', index_col=[0])


In [26]:
df.head()

,Symbol,Date,Open,High,Low,Close,Volume,FullDate,Time
0,AACG,04-01-2022,1.54,1.54,1.54,1.54,100,01-Apr-2022 09:14,09:14
1,AACG,04-01-2022,1.48,1.48,1.48,1.48,500,01-Apr-2022 09:26,09:26
2,AACG,04-01-2022,1.55,1.55,1.55,1.55,100,01-Apr-2022 09:27,09:27
3,AACG,04-01-2022,1.56,1.56,1.56,1.56,298,01-Apr-2022 09:30,09:30
4,AACG,04-01-2022,1.49,1.49,1.49,1.49,100,01-Apr-2022 09:32,09:32


In [27]:
# limit to Apple stock only for now

dfstk=df[df['Symbol'] == "AAPL"]
dfstk.head()
dfstk.to_csv('C:\\Users\\benro\\OneDrive\\Documents\\EODData\\StockProject_New\\AAPL_042022_062022.csv')

In [28]:
# Stock market is open 9:30-4:00, 6.5 hrs, or 390 minutes
dfstk['Time'].agg(['min', 'max'])
# These times 9am to 5pm represent some before and after hours trading
# 8 hours or 480 minutes


min    09:00
max    16:59
Name: Time, dtype: object

## Interval Comparisons

In [29]:
dfstk = pd.read_csv('C:\\Users\\benro\\OneDrive\\Documents\\EODData\\StockProject_New\\AAPL_042022_062022.csv', index_col=[0])
dfstk.head()

,Symbol,Date,Open,High,Low,Close,Volume,FullDate,Time
1144,AAPL,04-01-2022,173.65,173.67,173.55,173.60,18093,01-Apr-2022 09:00,09:00
1145,AAPL,04-01-2022,173.66,173.70,173.61,173.62,9482,01-Apr-2022 09:01,09:01
1146,AAPL,04-01-2022,173.66,173.66,173.40,173.40,20539,01-Apr-2022 09:02,09:02
1147,AAPL,04-01-2022,173.42,173.47,173.40,173.47,14775,01-Apr-2022 09:03,09:03
1148,AAPL,04-01-2022,173.44,173.44,173.30,173.30,18326,01-Apr-2022 09:04,09:04


In [30]:
# Let's first try to understand what's happening in the first 10, 20, and 30 minute of open market, our 'X' period
# For example, during the 'x' period:
# Was ClOSE higher or lower than the OPEN
# By how much
# Was that consistent for each 2 minute or 5 minute bar
# Was the OPEN higher or lower than the previous day CLOSE
# Was the CLOSE higher or lower than the previous day CLOSE
# Did VOLUME increase or decrease
# Was that consistent for each 2 minute or 5 minute bar
# Was VOLUME higher or lower than the x period yesterday
# What the price doing compared to the RSI  
# (https://www.wallstreetmojo.com/relative-strength-index/   
# RSI is calculated using the formula RSI = 100 – (100 / [1 + {14-Day Average Gain / 14-Day Average Loss}]))  


In [31]:
# Tick up, or green candle
dfstk['Green'] = dfstk['Close'].gt(dfstk['Open'])
dfstk.head(20)

,Symbol,Date,Open,High,Low,Close,Volume,FullDate,Time,Green
1144,AAPL,04-01-2022,173.65,173.67,173.55,173.60,18093,01-Apr-2022 09:00,09:00,False
1145,AAPL,04-01-2022,173.66,173.70,173.61,173.62,9482,01-Apr-2022 09:01,09:01,False
1146,AAPL,04-01-2022,173.66,173.66,173.40,173.40,20539,01-Apr-2022 09:02,09:02,False
1147,AAPL,04-01-2022,173.42,173.47,173.40,173.47,14775,01-Apr-2022 09:03,09:03,True
1148,AAPL,04-01-2022,173.44,173.44,173.30,173.30,18326,01-Apr-2022 09:04,09:04,False
1149,AAPL,04-01-2022,173.27,173.46,173.26,173.26,28783,01-Apr-2022 09:05,09:05,False
1150,AAPL,04-01-2022,173.44,173.61,173.40,173.50,36723,01-Apr-2022 09:06,09:06,True
1151,AAPL,04-01-2022,173.50,173.63,173.50,173.57,7217,01-Apr-2022 09:07,09:07,True
1152,AAPL,04-01-2022,173.60,173.71,173.26,173.50,35464,01-Apr-2022 09:08,09:08,False
1153,AAPL,04-01-2022,173.50,173.50,173.40,173.50,9329,01-Apr-2022 09:09,09:09,False


In [32]:
# Volume increase
# eq, ne, lt, gt :: ==, !=, <, >
dfstk['Int_vol_inc'] = dfstk.Volume.gt(dfstk.Volume.shift())
dfstk.head()

,Symbol,Date,Open,High,Low,Close,Volume,FullDate,Time,Green,Int_vol_inc
1144,AAPL,04-01-2022,173.65,173.67,173.55,173.60,18093,01-Apr-2022 09:00,09:00,False,False
1145,AAPL,04-01-2022,173.66,173.70,173.61,173.62,9482,01-Apr-2022 09:01,09:01,False,False
1146,AAPL,04-01-2022,173.66,173.66,173.40,173.40,20539,01-Apr-2022 09:02,09:02,False,True
1147,AAPL,04-01-2022,173.42,173.47,173.40,173.47,14775,01-Apr-2022 09:03,09:03,True,False
1148,AAPL,04-01-2022,173.44,173.44,173.30,173.30,18326,01-Apr-2022 09:04,09:04,False,True


In [35]:
# Simple Moving Averages
dfstk['SMA2'] = dfstk['Close'].rolling(2).mean()
dfstk['SMA5'] = dfstk['Close'].rolling(5).mean()
dfstk['SMA10'] = dfstk['Close'].rolling(10).mean()
dfstk['SMA20'] = dfstk['Close'].rolling(20).mean()
dfstk['SMA60'] = dfstk['Close'].rolling(60).mean()
dfstk['SMA120'] = dfstk['Close'].rolling(120).mean()
dfstk['SMA240'] = dfstk['Close'].rolling(240).mean()
dfstk['SMA1D'] = dfstk['Close'].rolling(390).mean() # Standard Trading Day
dfstk['SMA1De'] = dfstk['Close'].rolling(480).mean() # Extended Trading Day

dfstk['SMA2D'] = dfstk['Close'].rolling(780).mean() # 2 Standard Trading Days
dfstk['SMA2De'] = dfstk['Close'].rolling(960).mean() # 2 Extended Trading Days

dfstk['SMA3D'] = dfstk['Close'].rolling(1170).mean() # 3 Standard Trading Days
dfstk['SMA3De'] = dfstk['Close'].rolling(1440).mean() # 3 Extended Trading Days

dfstk['SMA5D'] = dfstk['Close'].rolling(1950).mean() # 5 Standard Trading Dayss
dfstk['SMA5De'] = dfstk['Close'].rolling(2400).mean() # 5 Extended Trading Days

dfstk['SMA10D'] = dfstk['Close'].rolling(3900).mean() # 10 Standard Trading Dayss
dfstk['SMA10De'] = dfstk['Close'].rolling(4800).mean() # 10 Extended Trading Days

dfstk['SMA20D'] = dfstk['Close'].rolling(7800).mean() # 20 Standard Trading Dayss
dfstk['SMA20De'] = dfstk['Close'].rolling(9600).mean() # 20 Extended Trading Days


dfstk.head()

,Symbol,Date,Open,High,Low,Close,Volume,FullDate,Time,Green,Int_vol_inc,SMA2,SMA5,SMA10,SMA20
1144,AAPL,04-01-2022,173.65,173.67,173.55,173.60,18093,01-Apr-2022 09:00,09:00,False,False,NaN,NaN,NaN,NaN
1145,AAPL,04-01-2022,173.66,173.70,173.61,173.62,9482,01-Apr-2022 09:01,09:01,False,False,173.610,NaN,NaN,NaN
1146,AAPL,04-01-2022,173.66,173.66,173.40,173.40,20539,01-Apr-2022 09:02,09:02,False,True,173.510,NaN,NaN,NaN
1147,AAPL,04-01-2022,173.42,173.47,173.40,173.47,14775,01-Apr-2022 09:03,09:03,True,False,173.435,NaN,NaN,NaN
1148,AAPL,04-01-2022,173.44,173.44,173.30,173.30,18326,01-Apr-2022 09:04,09:04,False,True,173.385,173.478,NaN,NaN


In [34]:
dfstk.head()

,Symbol,Date,Open,High,Low,Close,Volume,FullDate,Time,Green,Int_vol_inc,SMA2,SMA5,SMA10,SMA20
1144,AAPL,04-01-2022,173.65,173.67,173.55,173.60,18093,01-Apr-2022 09:00,09:00,False,False,NaN,NaN,NaN,NaN
1145,AAPL,04-01-2022,173.66,173.70,173.61,173.62,9482,01-Apr-2022 09:01,09:01,False,False,173.610,NaN,NaN,NaN
1146,AAPL,04-01-2022,173.66,173.66,173.40,173.40,20539,01-Apr-2022 09:02,09:02,False,True,173.510,NaN,NaN,NaN
1147,AAPL,04-01-2022,173.42,173.47,173.40,173.47,14775,01-Apr-2022 09:03,09:03,True,False,173.435,NaN,NaN,NaN
1148,AAPL,04-01-2022,173.44,173.44,173.30,173.30,18326,01-Apr-2022 09:04,09:04,False,True,173.385,173.478,NaN,NaN


In [36]:
# Exponential Moving Averages
dfstk['EWMA2'] = dfstk['Close'].ewm(span=2).mean()
dfstk['EWMA5'] = dfstk['Close'].ewm(span=5).mean()
dfstk['EWMA10'] = dfstk['Close'].ewm(span=10).mean()
dfstk['EWMA20'] = dfstk['Close'].ewm(span=20).mean()
dfstk['EWMA60'] = dfstk['Close'].ewm(span=60).mean()
dfstk['EWMA120'] = dfstk['Close'].ewm(span=120).mean()
dfstk['EWMA240'] = dfstk['Close'].ewm(span=240).mean()

dfstk['EWMA1D'] = dfstk['Close'].ewm(span=390).mean() # Standard Trading Day
dfstk['EWMA1De'] = dfstk['Close'].ewm(span=480).mean() # Extended Trading Day

dfstk['EWMA2D'] = dfstk['Close'].ewm(span=780).mean() # 2 Standard Trading Days
dfstk['EWMA2De'] = dfstk['Close'].ewm(span=960).mean() # 2 Extended Trading Days

dfstk['EWMA3D'] = dfstk['Close'].ewm(span=1170).mean() # 3 Standard Trading Days
dfstk['EWMA3De'] = dfstk['Close'].ewm(span=1440).mean() # 3 Extended Trading Days
 
dfstk['EWMA5D'] = dfstk['Close'].ewm(span=1950).mean() # 5 Standard Trading Days
dfstk['EWMA5De'] = dfstk['Close'].ewm(span=2400).mean() # 5 Extended Trading Days

dfstk['EWMA10D'] = dfstk['Close'].ewm(span=3900).mean() # 10 Standard Trading Days
dfstk['EWMA10De'] = dfstk['Close'].ewm(span=4880).mean() # 10 Extended Trading Days

dfstk['EWMA20D'] = dfstk['Close'].ewm(span=7800).mean() # 20 Standard Trading Days
dfstk['EWMA20De'] = dfstk['Close'].ewm(span=9600).mean() # 20 Extended Trading Days

dfstk.head()

,Symbol,Date,Open,High,Low,Close,Volume,FullDate,Time,Green,Int_vol_inc,SMA2,SMA5,SMA10,SMA20,EWMA2,EWMA5,EWMA10,EWMA20
1144,AAPL,04-01-2022,173.65,173.67,173.55,173.60,18093,01-Apr-2022 09:00,09:00,False,False,NaN,NaN,NaN,NaN,173.600000,173.600000,173.600000,173.600000
1145,AAPL,04-01-2022,173.66,173.70,173.61,173.62,9482,01-Apr-2022 09:01,09:01,False,False,173.610,NaN,NaN,NaN,173.615000,173.612000,173.611000,173.610500
1146,AAPL,04-01-2022,173.66,173.66,173.40,173.40,20539,01-Apr-2022 09:02,09:02,False,True,173.510,NaN,NaN,NaN,173.466154,173.511579,173.526179,173.533206
1147,AAPL,04-01-2022,173.42,173.47,173.40,173.47,14775,01-Apr-2022 09:03,09:03,True,False,173.435,NaN,NaN,NaN,173.468750,173.494308,173.507671,173.514959
1148,AAPL,04-01-2022,173.44,173.44,173.30,173.30,18326,01-Apr-2022 09:04,09:04,False,True,173.385,173.478,NaN,NaN,173.355785,173.419716,173.448054,173.462962


In [49]:
# MACD & Signal (Moving average convergence/divergence)
# If the MACD crosses above the Signal, it can be considered a bullish (BUY) signal. 
# If the MACD crosses below the Signal, it can be considered a bearish (SELL) signal. 

dfstk['MACD'] = dfstk['Close'].ewm(span=12).mean() - dfstk['Close'].ewm(span=26).mean()
dfstk['MACD_Sig'] = dfstk['MACD'].ewm(span=9).mean()
dfstk['MACD_Buy'] = dfstk['MACD'] > dfstk['MACD_Sig']
dfstk.head()

,Symbol,Date,Open,High,Low,Close,Volume,FullDate,Time,Green,...,SMA10,SMA20,EWMA2,EWMA5,EWMA10,EWMA20,MACD,MACD_Sig,RSI,MACD_Buy
1144,AAPL,04-01-2022,173.65,173.67,173.55,173.60,18093,01-Apr-2022 09:00,09:00,False,...,NaN,NaN,173.600000,173.600000,173.600000,173.600000,0.000000,0.000000,NaN,False
1145,AAPL,04-01-2022,173.66,173.70,173.61,173.62,9482,01-Apr-2022 09:01,09:01,False,...,NaN,NaN,173.615000,173.612000,173.611000,173.610500,0.000449,0.000249,NaN,True
1146,AAPL,04-01-2022,173.66,173.66,173.40,173.40,20539,01-Apr-2022 09:02,09:02,False,...,NaN,NaN,173.466154,173.511579,173.526179,173.533206,-0.006250,-0.002415,NaN,False
1147,AAPL,04-01-2022,173.42,173.47,173.40,173.47,14775,01-Apr-2022 09:03,09:03,True,...,NaN,NaN,173.468750,173.494308,173.507671,173.514959,-0.006617,-0.003838,NaN,False
1148,AAPL,04-01-2022,173.44,173.44,173.30,173.30,18326,01-Apr-2022 09:04,09:04,False,...,NaN,NaN,173.355785,173.419716,173.448054,173.462962,-0.013441,-0.006695,NaN,False


In [50]:
# RSI and upper/lower limits
# with help from https://www.roelpeters.be/many-ways-to-calculate-the-rsi-in-python-pandas/
# low RSI levels, <30, indicate oversold conditions(buy signal). Hhigh RSI levels, >70, indicate overbought conditions (sell signal).

def rsi(df, periods = 14, ema = True):
    """
    Returns a pd.Series with the relative strength index.
    """
    close_delta = df['Close'].diff()

    # Make two series: one for lower closes and one for higher closes
    up = close_delta.clip(lower=0)
    down = -1 * close_delta.clip(upper=0)
    
    if ema == True:
        # Use exponential moving average
        ma_up = up.ewm(com = periods - 1, adjust=True, min_periods = periods).mean()
        ma_down = down.ewm(com = periods - 1, adjust=True, min_periods = periods).mean()
    else:
        # Use simple moving average
        ma_up = up.rolling(window = periods, adjust=False).mean()
        ma_down = down.rolling(window = periods, adjust=False).mean()
        
    rsi = ma_up / ma_down
    rsi = 100 - (100/(1 + rsi))
    return rsi


dfstk['RSI'] = rsi(dfstk)
dfstk['RSI_Buy'] = dfstk['RSI'] < 30
dfstk.head()

,Symbol,Date,Open,High,Low,Close,Volume,FullDate,Time,Green,...,SMA20,EWMA2,EWMA5,EWMA10,EWMA20,MACD,MACD_Sig,RSI,MACD_Buy,RSI_Buy
1144,AAPL,04-01-2022,173.65,173.67,173.55,173.60,18093,01-Apr-2022 09:00,09:00,False,...,NaN,173.600000,173.600000,173.600000,173.600000,0.000000,0.000000,NaN,False,False
1145,AAPL,04-01-2022,173.66,173.70,173.61,173.62,9482,01-Apr-2022 09:01,09:01,False,...,NaN,173.615000,173.612000,173.611000,173.610500,0.000449,0.000249,NaN,True,False
1146,AAPL,04-01-2022,173.66,173.66,173.40,173.40,20539,01-Apr-2022 09:02,09:02,False,...,NaN,173.466154,173.511579,173.526179,173.533206,-0.006250,-0.002415,NaN,False,False
1147,AAPL,04-01-2022,173.42,173.47,173.40,173.47,14775,01-Apr-2022 09:03,09:03,True,...,NaN,173.468750,173.494308,173.507671,173.514959,-0.006617,-0.003838,NaN,False,False
1148,AAPL,04-01-2022,173.44,173.44,173.30,173.30,18326,01-Apr-2022 09:04,09:04,False,...,NaN,173.355785,173.419716,173.448054,173.462962,-0.013441,-0.006695,NaN,False,False


In [54]:
dfstk.head(20)

,Symbol,Date,Open,High,Low,Close,Volume,FullDate,Time,Green,...,SMA20,EWMA2,EWMA5,EWMA10,EWMA20,MACD,MACD_Sig,RSI,MACD_Buy,RSI_Buy
1144,AAPL,04-01-2022,173.65,173.67,173.55,173.60,18093,01-Apr-2022 09:00,09:00,False,...,NaN,173.600000,173.600000,173.600000,173.600000,0.000000,0.000000,NaN,False,False
1145,AAPL,04-01-2022,173.66,173.70,173.61,173.62,9482,01-Apr-2022 09:01,09:01,False,...,NaN,173.615000,173.612000,173.611000,173.610500,0.000449,0.000249,NaN,True,False
1146,AAPL,04-01-2022,173.66,173.66,173.40,173.40,20539,01-Apr-2022 09:02,09:02,False,...,NaN,173.466154,173.511579,173.526179,173.533206,-0.006250,-0.002415,NaN,False,False
1147,AAPL,04-01-2022,173.42,173.47,173.40,173.47,14775,01-Apr-2022 09:03,09:03,True,...,NaN,173.468750,173.494308,173.507671,173.514959,-0.006617,-0.003838,NaN,False,False
1148,AAPL,04-01-2022,173.44,173.44,173.30,173.30,18326,01-Apr-2022 09:04,09:04,False,...,NaN,173.355785,173.419716,173.448054,173.462962,-0.013441,-0.006695,NaN,False,False
1149,AAPL,04-01-2022,173.27,173.46,173.26,173.26,28783,01-Apr-2022 09:05,09:05,False,...,NaN,173.291841,173.361353,173.399210,173.420147,-0.019002,-0.010031,NaN,False,False
1150,AAPL,04-01-2022,173.44,173.61,173.40,173.50,36723,01-Apr-2022 09:06,09:06,True,...,NaN,173.430677,173.410442,173.423496,173.435245,-0.011368,-0.010369,NaN,False,False
1151,AAPL,04-01-2022,173.50,173.63,173.50,173.57,7217,01-Apr-2022 09:07,09:07,True,...,NaN,173.523573,173.465787,173.456827,173.458538,-0.002737,-0.008535,NaN,True,False
1152,AAPL,04-01-2022,173.60,173.71,173.26,173.50,35464,01-Apr-2022 09:08,09:08,False,...,NaN,173.507857,173.477496,173.466220,173.465189,-0.000192,-0.006608,NaN,True,False
1153,AAPL,04-01-2022,173.50,173.50,173.40,173.50,9329,01-Apr-2022 09:09,09:09,False,...,NaN,173.502619,173.485130,173.473315,173.470431,0.001617,-0.004765,NaN,True,False


In [55]:
dfstk.to_csv('C:\\Users\\benro\\OneDrive\\Documents\\EODData\\StockProject_New\\AAPL_042022_062022_updated.csv')